# Imbalance Data

In [1]:
import pandas as pd
import numpy  as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import random
from sklearn.utils import resample

In [2]:
n = 5000
skip = sorted(random.sample(range(1,319795),319795-n))
DF = pd.read_csv('../../datasets/heart_2020_cleaned.csv',skiprows=skip).reset_index().rename(columns={'index':'id'})

In [3]:
CatCols = [
    'Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','AgeCategory','Race','Diabetic',
    'PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer'
]
NumCols = [
    'BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime'
]

X = DF[DF.columns.difference(['id','HeartDisease'])]
Y = DF['HeartDisease']

In [4]:
X.head()

,AgeCategory,AlcoholDrinking,Asthma,BMI,Diabetic,DiffWalking,GenHealth,KidneyDisease,MentalHealth,PhysicalActivity,PhysicalHealth,Race,Sex,SkinCancer,SleepTime,Smoking,Stroke
0,65-69,No,No,35.51,Yes,Yes,Fair,No,0.0,Yes,10.0,White,Male,No,8.0,Yes,No
1,80 or older,No,No,28.29,No,Yes,Poor,No,3.0,No,30.0,White,Female,Yes,4.0,No,Yes
2,80 or older,No,Yes,30.61,No,Yes,Fair,No,8.0,Yes,0.0,White,Female,No,9.0,No,No
3,75-79,No,No,27.50,No,No,Fair,No,0.0,Yes,4.0,Black,Male,No,8.0,Yes,Yes
4,75-79,No,No,34.70,Yes,No,Fair,No,0.0,No,2.0,White,Female,Yes,10.0,Yes,Yes


In [5]:
# Drop rows with out of rage BMI
BMI_Out_Of_Range = X[X['BMI']>50]
X = X.drop(BMI_Out_Of_Range.index,errors='ignore')

In [6]:
# CatCols = [
#     'Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','Race','Diabetic',
#     'PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer'
# ]
# NumCols = [
#     'BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime','AgeCategory'
# ]

In [7]:
# _1 = X[X['AgeCategory']!='80 or older']['AgeCategory'].str.split('-').apply(
#     lambda x : (int(x[0])+int(x[1]))/2
# )
# _2 = X[X['AgeCategory']=='80 or older']['AgeCategory'].str.split(' or ').apply(
#     lambda x : 80
# )
# X.loc[_1.index,'AgeCategory'] = _1
# X.loc[_2.index,'AgeCategory'] = _2
# del _1,_2

In [8]:
# X['AgeCategory'] = X['AgeCategory'].astype(np.int64) 

In [9]:
# CatCols = [
#     'Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','Race','Diabetic',
#     'PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer'
# ]
# NumCols = [
#     'BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime','AgeCategory'
# ]

In [10]:
# Drop rows with out of rage SleepTime
X = X.drop(X[X['SleepTime']>16]['SleepTime'].index,axis=0)

# <font color="Green">One Hot Encode Categorical Label :</font>

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def OHC_transformer_single_column(DF,columns):
    
    for column_name in columns:
        label_encoder = LabelEncoder()
        encoded = label_encoder.fit_transform(DF[column_name])
        labels =pd.Series(label_encoder.classes_)

        clmt = ColumnTransformer([(column_name,OneHotEncoder(),[0])], remainder='passthrough')
        clmt_fited_data = clmt.fit_transform(DF[[column_name]]) 
        clmt_fited_data = clmt_fited_data if type(clmt_fited_data)== np.ndarray else clmt_fited_data.toarray()  
        NDF = pd.DataFrame(clmt_fited_data,columns=[f'{column_name}_{x}' for x in labels])
        
        merged = DF.merge(NDF, left_index=True,right_index=True)
        merged = merged.drop(columns=[column_name])
        DF = merged
    return DF

In [12]:
X_OHC = OHC_transformer_single_column(X,CatCols) 
Y = DF.iloc[X_OHC.index]['HeartDisease']
Y_label_encoder = LabelEncoder()
Y_OHC = Y_label_encoder.fit_transform(Y)

In [13]:
NDF = pd.concat([X,Y],axis=1)
NDF_OHC = pd.concat([X_OHC,pd.DataFrame(Y_OHC,columns=['HeartDisease'])],axis=1)

 # <font color="Green">Normalize  :</font>

In [14]:
X_OHC_Norm = X_OHC.copy()
ss = StandardScaler()
X_OHC_Norm[NumCols] = ss.fit_transform(X_OHC[NumCols])

In [15]:
X_OHC_Norm.shape

(4410, 50)

In [31]:
NDF = pd.concat([X_OHC_Norm,pd.DataFrame(Y_OHC,columns=['HeartDisease'])],axis=1)

In [35]:
X_OHC_Norm

,BMI,MentalHealth,PhysicalHealth,SleepTime,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,Stroke_Yes,...,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,1.246379,-0.499099,0.838773,0.644847,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.030586,-0.131649,3.368686,-2.199639,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.421257,0.480768,-0.426184,1.355968,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,-0.102444,-0.499099,0.079798,0.644847,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.109982,-0.499099,-0.173193,2.067090,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4445,-2.015381,-0.376616,-0.426184,-0.066275,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4446,0.934854,-0.499099,3.368686,0.644847,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4447,0.394314,-0.499099,-0.173193,1.355968,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4448,0.933170,2.562986,1.091764,-0.777396,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


# <font color="Green">Imbalance Data  :</font>

In [16]:
# from imblearn.over_sampling import RandomOverSampler
# toml =RandomOverSampler(random_state=3020)
# X_imb, Y_imb = toml.fit_resample(X_OHC_Norm,pd.DataFrame(Y_OHC,columns=['HeartDisease']))
# DF_imb = pd.concat([X_imb,Y_imb],axis=1)

In [24]:
df_majority_0 = NDF[(NDF['HeartDisease']==0)] 
df_minority_1 = NDF[(NDF['HeartDisease']==1)] 

df_minority_upsampled = resample(df_minority_1, 
                                 replace=True,    
                                 n_samples= df_majority_0.shape[0], 
                                 random_state=42) 

DF_imb = pd.concat([df_minority_upsampled, df_majority_0])


In [30]:

DF_imb

,BMI,MentalHealth,PhysicalHealth,SleepTime,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,Stroke_Yes,...,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes,HeartDisease
1257,1.153764,-0.499099,3.368686,2.067090,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4404,1.108298,-0.499099,-0.173193,-1.488517,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3378,-2.069267,-0.499099,-0.426184,1.355968,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
1392,0.177088,-0.499099,-0.426184,-0.066275,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
867,1.232908,1.215668,-0.426184,0.644847,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


 # <font color="Green">Modeling  :</font>

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(DF_imb.iloc[:,:-1],DF_imb.iloc[:,-1],random_state=3020)

In [28]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(
    random_state=3020,
    n_estimators=3000,
    verbose=True
)

random_forest_classifier.fit(X_train,Y_train)
# random_forest_classifier.score(X_test,Y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [29]:
Y_predicted = random_forest_classifier.predict(X_test)
print(metrics.classification_report(Y_test,Y_predicted))
print(metrics.confusion_matrix(Y_test,Y_predicted))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').